# Módulo MACHINE LEARNING
___

## Importamos librerías

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

## Carga dataset para entrenamiento

In [19]:
df_ml = pd.read_csv("../datasets/df_ml.csv")

## Función Entrenamiento

In [20]:
def train_random_forest(df_ml):
    # Seleccionar características de entrada (X) y variable objetivo (y)
    X = df_ml[['day_week', 'Hour', 'location_id']]
    y = df_ml['optimization']

    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Inicializar el modelo de Random Forests
    model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train, y_train)

    # Evaluar el modelo con los datos de prueba
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Mostrar las métricas de evaluación
    print(f"Error cuadrático medio (MSE): {mse}")
    print(f"Coeficiente de determinación (R^2): {r2}")

    return model

## Función de Uso

In [21]:
def get_top_5_location_id(date, hour, model, df_ml):
    # Convertir la fecha a day_week
    date = pd.to_datetime(date)
    day_week = date.dayofweek

    # Crear un DataFrame con todas las combinaciones de location_id, day_week, y Hour
    unique_locations = df_ml['location_id'].unique()
    test_data = pd.DataFrame({
        'location_id': unique_locations,
        'day_week': [day_week] * len(unique_locations),
        'Hour': [hour] * len(unique_locations)
    })

    test_data = test_data[['day_week', 'Hour', 'location_id']]

    # Predecir el valor de optimization para todas las combinaciones
    predictions = model.predict(test_data)

    # Añadir las predicciones al DataFrame
    test_data['optimization'] = predictions

    # Ordenar el DataFrame por la columna optimization en orden descendente
    test_data_sorted = test_data.sort_values(by='optimization', ascending=False)

    # Seleccionar los 5 mejores location_id
    top_5_locations = test_data_sorted.head(5)['location_id']

    # Retornar los 5 mejores location_id
    return list(top_5_locations)

## Entrenamos modelo

In [22]:
# Entrenar el modelo de Random Forests con df_ml
model = train_random_forest(df_ml)

Error cuadrático medio (MSE): 10.829733289956579
Coeficiente de determinación (R^2): 0.9845221583079596


## Ejemplo de uso

In [23]:
date = "2024-01-01"  # Fecha de ejemplo
hour = 5  # Hora de ejemplo

top_5_locations = get_top_5_location_id(date, hour, model, df_ml)
print("Top 5 location_id:", top_5_locations)

Top 5 location_id: [132, 48, 230, 68, 79]
